# Tutorial 9

In this tutorial, we will explore some ways to optimise input data pieplines. Begin by running the code below, then follow the instructions in the next section.

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import models, layers, optimizers, losses
import PIL
import os
import time

In [ ]:
# Loading in an image dataset, this one includes images of different types of
# flowers
import pathlib
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)
roses = list(data_dir.glob('roses/*'))
PIL.Image.open(str(roses[0]))


In [ ]:
# Create a dataset from the file names of the images and make a list of the
# class names
list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*'))
class_names = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

In [ ]:
# First function for processing images and labels. This one opens and decodes
# the images, and converts the label from a string into an interger
def process_image_label(filename):
    parts = tf.strings.split(filename, os.sep)
    one_hot = parts[-2] == class_names
    label = tf.argmax(one_hot)

    image = tf.io.read_file(filename)
    image = tf.io.decode_jpeg(image)
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, [128, 128])
    image = (image / 255.0)
    return image, label

In [ ]:
# An augmentation function which will manipulate images randomly on each epoch
def augment(image, label, seed):
    # We are using the "stateless" random functions, so we need to generate
    # random seeds
    new_seed = tf.random.experimental.stateless_split(seed, num=1)[0, :]
    image = tf.image.stateless_random_flip_left_right(
        image, seed)
    image = tf.image.stateless_random_brightness(
        image, max_delta=0.5, seed=new_seed)
    image = tf.image.stateless_random_hue(
        image, 0.1, seed)
    image = tf.image.stateless_random_saturation(
        image, 0.5, 1.0, seed)

    image = tf.clip_by_value(image, 0, 1)
    return image, label


In [ ]:
# Create a wrapper function for updating seeds.
def f(x, y):
  seed = rng.make_seeds(2)[0]
  image, label = augment(x, y, seed)
  return image, label

In [ ]:
# Create a random generator.
rng = tf.random.Generator.from_seed(123, alg='philox')

In [ ]:
### YOU'LL NEED TO COPY THIS BLOCK DOWN BELOW AND EDIT IT

# Create the basic datasets. Images are split into a training and validation set
val_size = int(image_count * 0.2)
train_ds = list_ds.skip(val_size)
val_ds = list_ds.take(val_size)

# Training images are parsed with the processing and augmentation functions
train_ds = (
    train_ds
    .shuffle(1000)
    .map(process_image_label)
    .map(f)
    .batch(32)
)

# Validation images are only parsed with the processing function
val_ds = (
    val_ds
    .map(process_image_label)
    .batch(32)
)

In [ ]:
# A fairly simple CNN, nothing fancy going on here
num_classes = len(class_names)

model = models.Sequential([
    layers.Input(shape=(128,128,3)),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [ ]:
# Training the model. We don't care so much about the results, more how long it
# takes to process two epochs
epochs=2
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)


# Exercise 1

Copy the code block that defines the datasets from above. In this exercise, you will be changing the dataset definition to improve the training times. For each experiment, define the new datasets, then run for two epochs and note the time it takes to process both.

Start by adjusting the batch size—see what happens when you alter the batch size from 16 images up to 256 images, in powers of 2.

# Exercise 2

Using the [Tensorflow Data Performance Guide](https://www.tensorflow.org/guide/data_performance#overview) as a reference, make edits to the data pipeline one change at a time, and see how it affects the training times. Consider how the order of the dataset transformations might affect the training process.

Your final pipeline could make use of the following methods (not necessarily in this order):
```
.prefetch
.map
.cache
```

You can make use of parallel processing in the data pipeline by using the argument `num_parallel_calls=tf.data.AUTOTUNE` in most of the dataset methods.

# Exercise 3

Use the TensorFlow profiler to explore the performance of the model on the GPU.

In [ ]:
!pip install -U tensorboard_plugin_profile

In [ ]:
from datetime import datetime
# Create a TensorBoard callback
logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")

# You'll need to edit the profile_batch here so that it profiles 10 batches
# in the second epoch of your training
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = (50,60))


In [ ]:
model.fit(train_ds,
          epochs=2,
          validation_data=val_ds,
          callbacks = [tboard_callback])

In [ ]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [ ]:
# Launch TensorBoard and navigate to the Profile tab to view performance profile
%tensorboard --logdir=logs